In [315]:
from selenium import webdriver
from datetime import datetime
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException


# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")


# URL of the webpage containing the dropdown menu
url = f"https://www.basketball-reference.com/leagues/NBA_2024_games.html"
driver.get(url)

time.sleep(2)

In [316]:
# Find months to filter data from
month_filter = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]')))

# Get link to each month
months = month_filter.find_elements(By.TAG_NAME, 'a')

# Get current date
current_date = datetime.now().strftime('%a, %b %d, %Y')

table_data = []
for i in range(len(months)):
    # Re-locate the element with the months to avoid stale element reference
    month_filter = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]')))
    months = month_filter.find_elements(By.TAG_NAME, 'a')
    
    # Get data for each month
    month = months[i].text
    print(f"Currently gathering data for {month}")
    
    # Change month to lowercase 
    month = months[i].text.lower()
    
    # Loop through all months
    url = f"https://www.basketball-reference.com/leagues/NBA_2024_games-{month}.html"
    driver.get(url)
    
    # Find the table on the page
    table = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="schedule"]'))
    )

    # Get table headers
    thead = table.find_element(By.TAG_NAME, 'thead')

    # Get the table body
    tbody = table.find_elements(By.TAG_NAME, 'tbody')


    # Get data from each row in the table
    rows = [tr.find_elements(By.TAG_NAME, 'tr') for tr in tbody][0]
    for row in rows:
    
        # Check for first instance of current date
        if current_date not in row.text:
            print(row.text)
        else:
            # Close the browser when done
            driver.quit()




Currently gathering data for October
Tue, Oct 24, 2023 7:30p Los Angeles Lakers 107 Denver Nuggets 119 Box Score 19,842 Ball Arena
Tue, Oct 24, 2023 10:00p Phoenix Suns 108 Golden State Warriors 104 Box Score 18,064 Chase Center
Wed, Oct 25, 2023 7:00p Houston Rockets 86 Orlando Magic 116 Box Score 18,846 Amway Center
Wed, Oct 25, 2023 7:00p Boston Celtics 108 New York Knicks 104 Box Score 19,812 Madison Square Garden (IV)
Wed, Oct 25, 2023 7:00p Washington Wizards 120 Indiana Pacers 143 Box Score 16,004 Gainbridge Fieldhouse
Wed, Oct 25, 2023 7:00p Atlanta Hawks 110 Charlotte Hornets 116 Box Score 16,129 Spectrum Center
Wed, Oct 25, 2023 7:30p Detroit Pistons 102 Miami Heat 103 Box Score 19,695 Kaseya Center
Wed, Oct 25, 2023 7:30p Minnesota Timberwolves 94 Toronto Raptors 97 Box Score 19,800 Scotiabank Arena
Wed, Oct 25, 2023 7:30p Cleveland Cavaliers 114 Brooklyn Nets 113 Box Score 17,931 Barclays Center
Wed, Oct 25, 2023 8:00p New Orleans Pelicans 111 Memphis Grizzlies 104 Box Scor

MaxRetryError: HTTPConnectionPool(host='localhost', port=56428): Max retries exceeded with url: /session/e0ea6bfca894a9d7e7ff56dfe93065ed/element/1BD1FC6B465D7E0EB023629762C32CCD_element_685/text (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020B8D3B2370>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))